# scouter

In [ ]:
import cv2
import torch
import torchvision.transforms as T
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize pre-trained SSD model
model = ssdlite320_mobilenet_v3_large(pretrained=True)
model.eval()

# If a GPU is available, move the model to GPU
if torch.cuda.is_available():
    model = model.cuda()

# Define the transformation
transform = T.Compose([T.ToPILImage(), T.Resize((300, 300)), T.ToTensor()])

# ...（前のコードは変更なし）

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply the transformation and make prediction
    inp = transform(frame).unsqueeze(0)
    if torch.cuda.is_available():
        inp = inp.cuda()
    
    with torch.no_grad():
        prediction = model(inp)

    # Filter out 'person' class with a confidence threshold
    confidence_threshold = 0.5
    for i in range(len(prediction[0]['labels'])):
        if prediction[0]['labels'][i] == 1 and prediction[0]['scores'][i] >= confidence_threshold:  # 1 is the COCO id for 'person'
            box = prediction[0]['boxes'][i].cpu().int().numpy()
            score = prediction[0]['scores'][i].cpu().item()
            power_level = score * 1000  # Just an example, you can define your own formula

            # Draw bounding box and power level
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
            cv2.putText(frame, f"Power: {power_level:.2f}", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Display the frame
    cv2.imshow('Scouter', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\Manyan\anaconda3\envs\osac\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\Manyan\anaconda3\envs\osac\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSDLite320_MobileNet_V3_Large_Weights.COCO_V1`. You can also use `weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
